In [11]:
from ultralytics import YOLO
import cv2
import os

MODEL_PATH = r"C:\Users\MINA\Downloads\Smoke Fire.pt"
INPUT_VIDEO = r"C:\Users\MINA\Downloads\Advancing Fire Safety with YOLOv8_ Real-Time Fire and Smoke Segmentation.mp4"
OUTPUT_VIDEO = r"C:\Users\MINA\Downloads\Advancing_Fire_Output.mp4"

conf_thresh = 0.25
imgsz = 640

model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(INPUT_VIDEO)
if not cap.isOpened():
    raise SystemExit(f"Cannot open video: {INPUT_VIDEO}")

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))

frame_idx = 0
print("Starting inference...")

cv2.namedWindow("Output", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Output", 1280, 720)
#cv2.setWindowProperty("Output", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = model.predict(source=[img_rgb], imgsz=imgsz, conf=conf_thresh, verbose=False, save=False)
    r = results[0]

    if r.boxes is not None and len(r.boxes) > 0:
        for box in r.boxes:
            xyxy = box.xyxy.cpu().numpy().astype(int)[0]
            conf = float(box.conf.cpu().numpy()[0])
            cls = int(box.cls.cpu().numpy()[0]) if box.cls is not None else -1
            x1, y1, x2, y2 = xyxy
            label = f"{model.names.get(cls, cls)} {conf:.2f}" if hasattr(model, "names") else f"{cls} {conf:.2f}"

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, label, (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    if hasattr(r, "masks") and r.masks is not None:
        try:
            masks = r.masks.data.cpu().numpy()
            for m in masks:
                m_uint8 = (m * 255).astype('uint8')
                colored_mask = cv2.applyColorMap(m_uint8, cv2.COLORMAP_JET)
                alpha = 0.35
                colored_mask = cv2.resize(colored_mask, (w, h))
                mask_gray = cv2.resize(m_uint8, (w, h))
                mask_bool = mask_gray > 127
                frame[mask_bool] = cv2.addWeighted(frame, 1 - alpha, colored_mask, alpha, 0)[mask_bool]
        except Exception as e:
            pass

    out.write(frame)

    cv2.imshow("Output", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print("✅ Done. Saved to:", OUTPUT_VIDEO)
cap.release()
out.release()
cv2.destroyAllWindows()


Starting inference...
✅ Done. Saved to: C:\Users\MINA\Downloads\Advancing_Fire_Output.mp4
